In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

sample_data = True

data_directory = os.path.join('.', 'data')

loan_data_path = os.path.join(data_directory, 'Accepted_LoanData.csv')
loan = pd.read_csv(loan_data_path, low_memory=False, encoding='latin-1')

pd.set_option('display.max_columns', len(loan.columns))
loan.head(3)

pd.reset_option('display.max_columns')

# see all loan table column variables datatypes.
print(loan.dtypes)

# See shape, all columns and statistic values for each columns.
print(loan.describe())

print(loan.shape)
print(loan.info())

# If we're sampling, reduce the data set down.
if sample_data:
    loan = loan.sample(frac=0.1, replace=False, random_state=1234, axis=0)
    
print(loan.shape)
loan.info()

id                                 object
member_id                         float64
loan_amnt                         float64
funded_amnt                       float64
funded_amnt_inv                   float64
term                               object
int_rate                           object
installment                       float64
grade                              object
sub_grade                          object
emp_title                          object
emp_length                         object
home_ownership                     object
annual_inc                        float64
verification_status                object
issue_d                            object
loan_status                        object
pymnt_plan                         object
url                                object
desc                               object
purpose                            object
title                              object
zip_code                           object
addr_state                        

In [2]:
#loan['loan_amnt'].dropna(inplace=True)

# Only keep INDIVIDUAL and JOINT application types.
# DIRECT_PAY have no DTI values, and there was one 'Unknown'/NA that had garbage values.
loan = loan[(loan['application_type'] == 'INDIVIDUAL') | (loan['application_type'] == 'JOINT')]

loan.loc[loan['application_type']=='INDIVIDUAL', 'dti'] = loan['dti']
loan.loc[loan['application_type']=='JOINT', 'dti'] = loan['dti_joint']

loan.loc[loan['application_type']=='INDIVIDUAL', 'annual_inc'] = loan['annual_inc']
loan.loc[loan['application_type']=='JOINT', 'annual_inc'] = loan['annual_inc_joint']

# Drop the unneeded _joint columns now.
loan.drop(['dti_joint', 'annual_inc_joint'], axis=1, inplace=True)
loan['annual_inc'].fillna(0, inplace=True)


# ID and member ID have no predictive power.
loan.drop(['id', 'member_id'], axis=1, inplace=True)

# Location of the borrower shouldn't have any affect (if time permitted, could repeat the analysis
# with these left in and verify this).
loan.drop(['zip_code', 'addr_state'], axis=1, inplace=True)

# Employee titles aren't as useful as income or DTI ratio.
loan.drop(['emp_title'], axis=1, inplace=True)

# Don't need current loan_status as we are trying to predict the interest rate at the point in time.
# at which the loan is granted.
loan.drop(['loan_status'], axis=1, inplace=True)

# Payment plan has the same problem as loan_status: it is after the fact instead of a priori.
loan.drop(['pymnt_plan'], axis=1, inplace=True)

# The URL, description and title are non-numeric and not useful for predicting interest rates, so drop them.
loan.drop(['url', 'desc', 'title'], axis=1, inplace=True)

# Convert the earliest credit line to a numeric (months since first credit line).
from datetime import datetime

dt_now = datetime.now()
loan.earliest_cr_line.fillna(dt_now.strftime('%b-%Y'), inplace=True)
loan.earliest_cr_line = pd.to_datetime(loan.earliest_cr_line)

dttoday = dt_now.strftime('%Y-%m-%d')

loan.earliest_cr_line = loan.earliest_cr_line.apply(lambda x: 
                            (np.timedelta64((x - pd.Timestamp(dttoday)),'D').astype(int))/-365)

loan.earliest_cr_line = (round(loan.earliest_cr_line)).astype(int)

use_fico = True
if use_fico:
    # Use a range and mean instead of low FICO and high FICO.
    loan['fico_range'] = loan.fico_range_low.astype('str') + '-' + loan.fico_range_high.astype('str')
    loan['mean_fico'] = (loan.fico_range_low + loan.fico_range_high)/2
    loan.drop(['fico_range_low', 'fico_range_high', 'initial_list_status'], axis=1, inplace=True)

    # Use a range and mean instead of last low FICO and last high FICO.
    loan['last_fico_range'] = loan.last_fico_range_low.astype('str') + '-' + loan.last_fico_range_high.astype('str')
    loan['last_mean_fico'] = (loan.last_fico_range_low + loan.last_fico_range_high)/2
    loan.drop(['last_fico_range_high', 'last_fico_range_low', 'policy_code'], axis=1, inplace=True)

# Drop columns for properties that were added AFTER the loan was granted.
loan.drop(['out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 
           'grade', 'sub_grade', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 
           'collection_recovery_fee', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 
           'next_pymnt_d', 'last_credit_pull_d', 'funded_amnt_inv', 'installment', 
           'total_pymnt', 'total_pymnt_inv', 'total_rev_hi_lim', 'tot_coll_amt',
           'tot_cur_bal', 'acc_now_delinq'], axis=1, inplace=True)

# Drop columns for the number of accounts, since that assumes the borrower was granted a loan
# and could leak information.
loan.drop(['num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 
           'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 
           'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m'], axis=1, inplace=True)

# Drop columns for 'months since' accounts opened, since that assumes the borrower was granted loans
# and could leak information.
loan.drop(['mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
           'mths_since_recent_bc', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 
           'mths_since_recent_revol_delinq'], axis=1, inplace=True)

# Drop columns for the number of open accounts/trades, since that assumes the borrower was granted a loan
# and could leak information.
loan.drop(['open_acc', 'open_acc_6m', 'open_il_12m', 'open_il_24m', 'open_il_6m', 
           'open_rv_12m', 'open_rv_24m'], axis=1, inplace=True)

# Drop columns for the total credit limits, since that assumes the borrower was granted a loan
# and could leak information.
loan.drop(['tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit'], 
          axis=1, inplace=True)

##############
##############

# We will fill verification_status_joint using the value in verification_status as these are all individual applications and these values are not filled out.
loan['verification_status_joint'].fillna(loan['verification_status'], inplace=True)

# We will clean the emp_length column to use it in data exploration
loan['emp_length'] = loan.emp_length.str.replace('+', '')
loan['emp_length'] = loan.emp_length.str.replace('<', '')
loan['emp_length'] = loan.emp_length.str.replace('years', '')
loan['emp_length'] = loan.emp_length.str.replace('year', '')
loan['emp_length'] = loan.emp_length.str.replace('n/a', '0')

print("Unique values for emp_length:")
print(loan.emp_length.unique())

# Convert emp_length datatype to float as we are using it for plotting graphs in data exploration.
loan['emp_length'] = loan['emp_length'].astype('float64')


# Convert int_rate from string, but only if it isn't already numeric.
if loan.int_rate.dtype == 'float64':
    print("int_rate is already numeric, no need to convert it")

elif loan.int_rate.dtype == 'object':
    loan['int_rate'] = loan['int_rate'].map(lambda x: str(x).lstrip(' ').rstrip('% '))
    print(loan['int_rate'][0:5])
    loan['int_rate'] = loan['int_rate'].astype('float64')

    
# Convert revol_util from string, but only if it isn't already numeric.
if loan.revol_util.dtype == 'float64':
    print("revol_util is already numeric, no need to convert it")

elif loan.revol_util.dtype == 'object':
    loan['revol_util'] = loan['revol_util'].map(lambda x: str(x).lstrip(' ').rstrip('% '))
    print(loan['revol_util'][0:5])
    loan['revol_util'] = loan['revol_util'].astype('float64')


# For the remaining categorical variables we are just going to replace NaN with 'Unknown'.
strColumns = loan.select_dtypes(include=['object']).columns.values
loan[strColumns] = loan[strColumns].fillna('Unknown')

# See if any null value is present or not for object or not a number data type.
loan.select_dtypes(exclude=[np.number]).isnull().sum()

# Check number of missing values for each numeric column variable.
loan.select_dtypes(include=[np.number]).isnull().sum()

#loan['annual_inc_joint'].fillna(0, inplace=True)
#loan['income'] = loan['annual_inc'] + loan['annual_inc_joint']

#for i,r in loan.iterrows()
#loan['dti_joint'].fillna(0, inplace=True)
#loan['dti'] = loan['dti'] + loan['dti_joint']
#loan.drop(['application_type'], axis=1, inplace=True)

# The first columns that we are going to update are annual_inc_joint, dti_joint. 
# For individual accounts these are blank but we want to use the joint values 
# so we will populate these with the individual values for individual accounts.
#loan[loan['application_type'] != 'INDIVIDUAL']['annual_inc_joint'].isnull().sum()
#loan['annual_inc_joint'].fillna(loan['annual_inc'], inplace=True)
#loan['dti_joint'].fillna(loan['dti'], inplace=True)

# For the remaining missing values we are going to fix it by replacing any NaN values with the mean values.
strColumns = loan.select_dtypes(include=[np.number]).columns.values
loan[strColumns] = loan[strColumns].fillna(loan[strColumns].mean())

loan.select_dtypes(include=[np.number]).isnull().sum()

# Features below are being dropped due to their significantly high proportion of missing values or they are date values.
loan.drop(['mths_since_rcnt_il', 'total_bal_il', 'il_util', 'max_bal_bc', 'all_util', 
           'inq_fi', 'total_cu_tl', 'inq_last_12m', 'mths_since_last_record', 
           'mths_since_last_major_derog'], axis=1, inplace=True)

print(loan.info())

Unique values for emp_length:
['2 ' '4 ' '10 ' '6 ' ' 1 ' '1 ' '0' '7 ' '3 ' '8 ' '9 ' '5 ']
1029298     8.99
63319      10.99
1182611     9.49
204800     21.98
80587      10.99
Name: int_rate, dtype: object
1029298    82.8
63319      86.4
1182611    52.4
204800     77.1
80587      56.9
Name: revol_util, dtype: object
<class 'pandas.core.frame.DataFrame'>
Int64Index: 131893 entries, 1029298 to 857921
Data columns (total 38 columns):
loan_amnt                     131893 non-null float64
funded_amnt                   131893 non-null float64
term                          131893 non-null object
int_rate                      131893 non-null float64
emp_length                    131893 non-null float64
home_ownership                131893 non-null object
annual_inc                    131893 non-null float64
verification_status           131893 non-null object
issue_d                       131893 non-null object
purpose                       131893 non-null object
dti                         

In [3]:
# Check correlation of the columns.
cor = loan.corr()
cor.loc[:,:] = np.tril(cor, k=-1)
cor = cor.stack()
cor[(cor > 0.55) | (cor < -0.55)]

funded_amnt           loan_amnt     0.999548
bc_util               revol_util    0.818679
percent_bc_gt_75      revol_util    0.694352
                      bc_util       0.840269
pub_rec_bankruptcies  pub_rec       0.579223
tax_liens             pub_rec       0.761653
dtype: float64

In [4]:
# Drop columns that are correlated.
loan.drop(['funded_amnt', 'bc_util', 'percent_bc_gt_75', 'tax_liens', 'pub_rec_bankruptcies'], axis=1, inplace=True)

In [5]:
# save cleaned data to .csv
destination_filepath = os.path.join(data_directory, "Cleaned_AcceptedLoanData.csv")
loan.to_csv(destination_filepath, encoding="UTF-8")

loan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131893 entries, 1029298 to 857921
Data columns (total 33 columns):
loan_amnt                     131893 non-null float64
term                          131893 non-null object
int_rate                      131893 non-null float64
emp_length                    131893 non-null float64
home_ownership                131893 non-null object
annual_inc                    131893 non-null float64
verification_status           131893 non-null object
issue_d                       131893 non-null object
purpose                       131893 non-null object
dti                           131893 non-null float64
delinq_2yrs                   131893 non-null float64
earliest_cr_line              131893 non-null int32
inq_last_6mths                131893 non-null float64
mths_since_last_delinq        131893 non-null float64
pub_rec                       131893 non-null float64
revol_bal                     131893 non-null float64
revol_util                

In [6]:
print(loan['dti'].value_counts())
print(loan['dti'].isnull().sum())

print(loan['dti_joint'].isnull().sum())

print(loan['dti'].unique())

19.20    116
20.40    108
18.00    100
12.00     99
13.20     97
14.40     95
16.80     90
15.60     86
21.60     86
10.80     85
15.16     84
19.92     84
17.19     84
17.28     83
15.06     82
15.10     82
14.35     82
13.44     81
13.38     80
12.83     80
24.00     79
16.64     79
18.72     78
15.56     78
14.64     78
16.32     77
14.75     77
17.94     76
9.60      76
16.26     76
        ... 
39.06      2
0.85       2
46.52      1
39.15      1
0.13       1
38.48      1
36.73      1
38.10      1
43.91      1
35.11      1
38.71      1
0.07       1
39.34      1
0.12       1
1.72       1
38.39      1
35.70      1
39.36      1
40.08      1
39.59      1
0.31       1
1.75       1
1.06       1
39.82      1
0.03       1
0.05       1
36.01      1
38.22      1
37.57      1
0.18       1
Name: dti, dtype: int64
0


KeyError: 'dti_joint'